In [2]:
%pip install xgboost xgboost_ray

  Using cached xgboost-1.6.2-py3-none-manylinux2014_x86_64.whl (255.9 MB)
  Using cached xgboost_ray-0.1.15-py3-none-any.whl (138 kB)
  Using cached wrapt-1.15.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (75 kB)
  Using cached ray-2.3.0-cp37-cp37m-manylinux2014_x86_64.whl (58.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 40.9 MB/s eta 0:00:00:00:0100:01
  Using cached grpcio-1.51.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached msgpack-1.0.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (299 kB)
  Using cached distlib-0.3.6-py2.py3-none-any.whl (468 kB)
  Using cached filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Attempting uninstall: msgpack
    Found existing installation: msgpack 0.6.1
    Uninstalling msgpack-0.6.1:
      Successfully uninstalled msgpack-0.6.1
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
  

In [3]:
import json
import pandas as pd

import xgboost as xgb

import ray
ray.shutdown()

from ray import tune
from ray.air.checkpoint import Checkpoint
from ray.train.constants import TRAIN_DATASET_KEY

from ray.train.xgboost import XGBoostCheckpoint, XGBoostTrainer
from ray.air.config import ScalingConfig
from ray.data.preprocessor import Preprocessor

# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split

In [4]:
import multiprocessing
import ray

num_cpus = multiprocessing.cpu_count()

ray.shutdown()
address_info = ray.init(num_cpus=num_cpus - 1) #,log_dir='/tmp/ray_results/')


2023-03-14 04:30:33,619	WARNING services.py:1791 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 4044357632 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.00gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-03-14 04:30:33,744	INFO worker.py:1553 -- Started a local Ray instance.


In [5]:
########
# DATA #
########
# Read Parquet file to Ray Dataset
dataset = ray.data.read_parquet(
    './data/nyc-taxi/'
#    "s3://anyscale-training-data/intro-to-ray-air/nyc_taxi_2021.parquet"
#    "s3://dsoaws/intro-to-ray-air/nyc_taxi_2021.parquet"
#    's3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-all-years-multiple-files-100GB/'
)

# Split data into training and validation subsets
#train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Split datasets into blocks for parallel preprocessing
# `num_blocks` should be lower than number of cores in the cluster
train_dataset = dataset.repartition(num_blocks=96)
#valid_dataset = valid_dataset.repartition(num_blocks=5)

# Define a preprocessor to normalize the columns by their range
#preprocessor = MinMaxScaler(columns=["trip_distance", "trip_duration"])

############
# TRAINING #
############

# Create XGBoost trainer.
# During training, it will use `num_blocks` workers.
trainer = XGBoostTrainer(
    label_column="total_amount",
    num_boost_round=50,
    scaling_config=ScalingConfig(
        num_workers=num_cpus - 1,
        use_gpu=False,  # True for the GPU training, 1 GPU per worker
    ),
    params={
        "eta": "0.2",
        "gamma": "4",
        "max_depth": "5",
        "min_child_weight": "6",
#        "num_round": "50",
        "objective": "reg:squarederror",
        "subsample": "0.7",
        "verbosity": "2",
    },
    datasets={
        "train": train_dataset,
    },
#    preprocessor=preprocessor,
)

# Invoke training - this is computationally intensive operation
# The resulting object grants access to metrics, checkpoints, and errors
result = trainer.fit()

# Report results
print(f"train rmse = {result.metrics['train-rmse']}")
#print(f"valid acc = {1 - result.metrics['valid-error']:.4f}")
print(f"iteration = {result.metrics['training_iteration']}")

(_get_read_tasks pid=1139) /opt/conda/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:233: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=1139)   pq_ds.pieces, **prefetch_remote_args
(_get_read_tasks pid=1139) /opt/conda/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:311: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=1139)   num_files = len(self._pq_ds.pieces)
(_get_read_tasks pid=1139) /opt/conda/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:325: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=1139)   for idx in n

Parquet Files Sample: 100%|██████████| 2/2 [00:01<00:00,  1.87it/s]
(_get_read_tasks pid=1139) /opt/conda/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:263: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=1139)   np.array_split(self._pq_ds.pieces, parallelism),
(XGBoostTrainer pid=1586) 2023-03-14 04:30:39,023	WARNING plan.py:524 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for more details: https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune
(XGBoostTrainer pid=1586) 2023-03-14 04:30:39,032	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read] -> AllToAllOperator[repartition]


(autoscaler +16s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +16s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(autoscaler +52s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(autoscaler +1m27s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(autoscaler +2m2s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all clust

2023-03-14 05:13:05,369	WARNING tune.py:147 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-03-14 05:13:06,417	ERROR tune.py:794 -- Trials did not complete: [XGBoostTrainer_f764c_00000]
2023-03-14 05:13:06,418	INFO tune.py:799 -- Total run time: 2550.05 seconds (2550.01 seconds for the tuning loop).
2023-03-14 05:13:06,419	WARNING tune.py:805 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


KeyError: 'train-rmse'

# Resume from checkpoint

In [ ]:
# trainer = XGBoostTrainer(
#     scaling_config=scale_config,
#     label_column="target",
#     params=params,
#     num_boost_round=5,
#     datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
# )
# result = trainer.fit()
# checkpoint = XGBoostCheckpoint.from_checkpoint(result.checkpoint)
# xgb_model = checkpoint.get_model()
# assert get_num_trees(xgb_model) == 5

# Tuner

In [ ]:
trainer = XGBoostTrainer(
    scaling_config=scale_config,
    label_column="target",
    params={**params, **{"max_depth": 1}},
    datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
)

tune.run(
    trainer.as_trainable(),
    config={"params": {"max_depth": tune.randint(2, 4)}},
    num_samples=2,
)

# # Make sure original Trainer is not affected.
# assert trainer.params["max_depth"] == 1

In [ ]:
# # Move checkpoint to a different directory.
# checkpoint_dict = result.checkpoint.to_dict()
# checkpoint = Checkpoint.from_dict(checkpoint_dict)
# checkpoint_path = checkpoint.to_directory(tmpdir)
# resume_from = Checkpoint.from_directory(checkpoint_path)

# trainer = XGBoostTrainer(
#     scaling_config=scale_config,
#     label_column="target",
#     params=params,
#     num_boost_round=5,
#     datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#     resume_from_checkpoint=resume_from,
# )
# result = trainer.fit()
# checkpoint = XGBoostCheckpoint.from_checkpoint(result.checkpoint)
# model = checkpoint.get_model()
# assert get_num_trees(model) == 10


In [ ]:
# @pytest.mark.parametrize(
#     "freq_end_expected",
#     [
#         (4, True, 7),  # 4, 8, 12, 16, 20, 24, 25
#         (4, False, 6),  # 4, 8, 12, 16, 20, 24
#         (5, True, 5),  # 5, 10, 15, 20, 25
#         (0, True, 1),
#         (0, False, 0),
#     ],
# )
# def test_checkpoint_freq(ray_start_4_cpus, freq_end_expected):
#     freq, end, expected = freq_end_expected

#     train_dataset = ray.data.from_pandas(train_df)
#     valid_dataset = ray.data.from_pandas(test_df)
#     trainer = XGBoostTrainer(
#         run_config=ray.air.RunConfig(
#             checkpoint_config=ray.air.CheckpointConfig(
#                 checkpoint_frequency=freq, checkpoint_at_end=end
#             )
#         ),
#         scaling_config=scale_config,
#         label_column="target",
#         params=params,
#         num_boost_round=25,
#         datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#     )
#     result = trainer.fit()

#     # Assert number of checkpoints
#     assert len(result.best_checkpoints) == expected, str(
#         [
#             (metrics["training_iteration"], _cp._local_path)
#             for _cp, metrics in result.best_checkpoints
#         ]
#     )

#     # Assert checkpoint numbers are increasing
#     cp_paths = [cp._local_path for cp, _ in result.best_checkpoints]
#     assert cp_paths == sorted(cp_paths), str(cp_paths)


# def test_preprocessor_in_checkpoint(ray_start_4_cpus, tmpdir):
#     train_dataset = ray.data.from_pandas(train_df)
#     valid_dataset = ray.data.from_pandas(test_df)

#     class DummyPreprocessor(Preprocessor):
#         def __init__(self):
#             super().__init__()
#             self.is_same = True

#         def fit(self, dataset):
#             self.fitted_ = True

#         def _transform_pandas(self, df: "pd.DataFrame") -> "pd.DataFrame":
#             return df

#     trainer = XGBoostTrainer(
#         scaling_config=scale_config,
#         label_column="target",
#         params=params,
#         datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#         preprocessor=DummyPreprocessor(),
#     )
#     result = trainer.fit()

#     # Move checkpoint to a different directory.
#     checkpoint_dict = result.checkpoint.to_dict()
#     checkpoint = Checkpoint.from_dict(checkpoint_dict)
#     checkpoint_path = checkpoint.to_directory(tmpdir)
#     resume_from = Checkpoint.from_directory(checkpoint_path)

#     resume_from = XGBoostCheckpoint.from_checkpoint(resume_from)

#     model, preprocessor = resume_from.get_model(), resume_from.get_preprocessor()
#     assert get_num_trees(model) == 10
#     assert preprocessor.is_same
#     assert preprocessor.fitted_




In [ ]:

# train_dataset = ray.data.from_pandas(train_df)
# valid_dataset = ray.data.from_pandas(test_df)
# #with pytest.raises(KeyError, match=TRAIN_DATASET_KEY):
# XGBoostTrainer(
#     scaling_config=ScalingConfig(num_workers=2),
#     label_column="target",
#     params=params,
#     datasets={"valid": valid_dataset},
# )
# with pytest.raises(KeyError, match="dmatrix_params"):
#     XGBoostTrainer(
#         scaling_config=ScalingConfig(num_workers=2),
#         label_column="target",
#         params=params,
#         dmatrix_params={"data": {}},
#         datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#     )




In [ ]:
# def test_distributed_data_loading(ray_start_4_cpus):
#     """Checks that XGBoostTrainer does distributed data loading for Ray Datasets."""

#     class DummyXGBoostTrainer(XGBoostTrainer):
#         def _train(self, params, dtrain, **kwargs):
#             assert dtrain.distributed
#             return super()._train(params=params, dtrain=dtrain, **kwargs)

#     train_dataset = ray.data.from_pandas(train_df)

#     trainer = DummyXGBoostTrainer(
#         scaling_config=ScalingConfig(num_workers=2),
#         label_column="target",
#         params=params,
#         datasets={TRAIN_DATASET_KEY: train_dataset},
#     )

#     assert trainer.dmatrix_params[TRAIN_DATASET_KEY]["distributed"]
#     trainer.fit()
